In [98]:
from util_functs import *
from pathlib import Path
from IPython.display import Audio
import time
import os
import shutil
from os.path import isdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import imageio
import h5py

from keras.models import load_model
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%pylab inline

np.random.seed(1)

Populating the interactive namespace from numpy and matplotlib


C:\Users\isaaa\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['clip', 'fft']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [111]:
#Testing the model on the unseen test data. 

model_path = 'models/model_94t.h5'
audio_path = 'input_clips/'

X_test, y_test, ms_df_test = generate_data((34,50,4), ms_dir='test_ms/', csv_name='test.csv')

model = load_model(model_path)

predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1) + 1
predictions = predictions.reshape(len(ms_df_test),1)

diff_array = predictions - y_test
correct_preds = diff_array[diff_array == 0]

acc_percent = (len(correct_preds) / len(diff_array)) * 100
acc_percent

98.95833333333334

In [100]:
i=random.randint(len(ms_df_test))
prediction = np.argmax(model.predict(np.array([X_test[i]]))[0]) + 1

#img = imageio.imread(test_ms_dir + ms_df_test.iloc[i].fname)
#imgplot = plt.imshow(img)
print("For the image {}, the model predicts that the song is {}".format(ms_df_test.iloc[i].fname, str(prediction) + ": " + LABEL_DICT[prediction]))

#PLay clip associated with above image
filename = audio_path + (os.path.splitext(ms_df_test.iloc[i].fname)[0] + '.wav')
clip, sample_rate = librosa.load(filename, sr=None)
Audio(clip, rate=sample_rate)

For the image 3_I_clip338.png, the model predicts that the song is 3: One Direction - What Makes You Beautiful


The model achieves 97.7% accuracy on the test set, and the data was not cleaned! It seems like the CNN was able to adapt in the presence of bad data (containing low noise, non-song content).

In [117]:
## INSERT LOCATION OF CLIP HERE ##
your_clip_name = '3.mp3'
##################################

clip_length = 6
clip, sample_rate = librosa.load("your_recordings/" + your_clip_name, sr=None, duration=clip_length)
librosa.output.write_wav("your_audio_clips/" + os.path.splitext(your_clip_name)[0] + ".wav", clip, sample_rate)

generate_melspecs(input_dir='your_audio_clips/', ms_output="your_ms/")
assert(len(os.listdir("your_ms/")) != 0) 

X, _ = initialise_data(img_folder='your_ms/', image_shape=(34,50,4))
img_name = os.path.splitext(your_clip_name)[0] + '.png'
img = imageio.imread("your_ms/" + img_name)
X[0] = img

prediction = model.predict(np.array([X[0]]))[0] #softmax vector 
ind = np.argpartition(prediction, -2)[-2:] #get indices of top two preds

best_guess = ind[np.argsort(prediction[ind])][1] + 1
second_best = ind[np.argsort(prediction[ind])][0] + 1

#img = imageio.imread(test_ms_dir + ms_df_test.iloc[i].fname)
#imgplot = plt.imshow(img)
print("For the image {}, best guess: {}, 2nd best guess: {}".format(img_name, LABEL_DICT[best_guess], LABEL_DICT[second_best]))

#PLay clip associated with above image
filename = 'your_audio_clips/' + os.path.splitext(your_clip_name)[0] + ".wav"
clip, sample_rate = librosa.load(filename, sr=None)
Audio(clip, rate=sample_rate)

For the image 5-2.png, best guess: Michael Jackson - Thriller, 2nd best guess: Survivor - Eye of the Tiger


In [102]:
assert(os.listdir('test_ms/') not in os.listdir('input_ms/'))